In [3]:
from pathlib import Path
import pandas as pd

# Path to your data folder
data_folder = Path("../Data")

# Find all *_minute.csv files
csv_files = list(data_folder.glob("*_minute.csv"))

all_dfs = []
company_id_map = {}
next_company_id = 1

for file_path in csv_files:
    # Extract company name from filename: "360ONE_minute.csv" -> "360ONE"
    company_name = file_path.stem.split("_")[0]

    # Assign a unique serial number for each company
    if company_name not in company_id_map:
        company_id_map[company_name] = next_company_id
        next_company_id += 1

    company_id = company_id_map[company_name]

    # Read the CSV
    df = pd.read_csv(file_path)

    # Add the two required columns
    df["company_name"] = company_name
    df["company_number"] = company_id

    all_dfs.append(df)

# Concatenate all into a single DataFrame
final_df = pd.concat(all_dfs, ignore_index=True)

# Optional: inspect result
print(final_df.head())

                  date  open  high   low  close  volume company_name  \
0  2015-06-11 10:50:00  11.8  11.8  11.8   11.8     0.0    DELHIVERY   
1  2015-06-11 10:51:00  11.8  11.8  11.8   11.8     0.0    DELHIVERY   
2  2015-06-11 10:52:00  11.8  11.8  11.8   11.8     0.0    DELHIVERY   
3  2015-06-11 10:53:00  11.8  11.8  11.8   11.8     0.0    DELHIVERY   
4  2015-06-11 10:54:00  11.8  11.8  11.8   11.8     0.0    DELHIVERY   

   company_number  
0               1  
1               1  
2               1  
3               1  
4               1  


In [11]:
#len(final_df) # Total number of rows
distinct_counts = final_df.nunique() # Distinct values count of each column
distinct_counts

date               974800
open              1393075
high              1346095
low               1348832
close             1392612
volume            1181340
company_name          486
company_number        486
dtype: int64

In [12]:
# Count of rows for each company (using company_name)
rows_per_company = final_df.groupby("company_name").size().reset_index(name="row_count")
print("\nRows per company:")
print(rows_per_company)


Rows per company:
    company_name  row_count
0         360ONE     603745
1        3MINDIA     951599
2      AADHARHFC     111879
3       AARTIIND     968650
4          AAVAS     620975
..           ...        ...
481      YESBANK     969507
482         ZEEL     969516
483   ZENSARTECH     959393
484    ZFCVINDIA     930303
485    ZYDUSLIFE     969530

[486 rows x 2 columns]


In [13]:
# If your date column is not yet datetime, convert it once:
final_df["date"] = pd.to_datetime(final_df["date"])

# Start and end date for each company (by company_name)
company_date_range = (
    final_df
    .groupby("company_name")["date"]
    .agg(start_date="min", end_date="max")
    .reset_index()
)

print(company_date_range)

    company_name          start_date            end_date
0         360ONE 2015-02-02 09:15:00 2025-07-25 15:29:00
1        3MINDIA 2015-02-02 09:15:00 2025-07-25 15:29:00
2      AADHARHFC 2024-05-15 09:15:00 2025-07-25 15:29:00
3       AARTIIND 2015-02-02 09:15:00 2025-07-25 15:29:00
4          AAVAS 2018-10-08 09:44:00 2025-07-25 15:29:00
..           ...                 ...                 ...
481      YESBANK 2015-02-02 09:15:00 2025-07-25 15:29:00
482         ZEEL 2015-02-02 09:15:00 2025-07-25 15:29:00
483   ZENSARTECH 2015-02-02 09:15:00 2025-07-25 15:29:00
484    ZFCVINDIA 2015-02-02 09:15:00 2025-07-25 15:29:00
485    ZYDUSLIFE 2015-02-02 09:15:00 2025-07-25 15:29:00

[486 rows x 3 columns]


#### adding start and end timestamp for all companies

In [4]:


# Ensure timestamp column is datetime
final_df["date"] = pd.to_datetime(final_df["date"])

# Compute start and end per company and add as columns
final_df["company_start_timestamp"] = (
    final_df.groupby("company_name")["date"].transform("min")
)

final_df["company_end_timestamp"] = (
    final_df.groupby("company_name")["date"].transform("max")
)

# Optional: check result
final_df.head()

,date,open,high,low,close,volume,company_name,company_number,company_start_timestamp,company_end_timestamp
0,2015-06-11 10:50:00,11.8,11.8,11.8,11.8,0.0,DELHIVERY,1,2015-06-11 10:50:00,2025-07-25 15:29:00
1,2015-06-11 10:51:00,11.8,11.8,11.8,11.8,0.0,DELHIVERY,1,2015-06-11 10:50:00,2025-07-25 15:29:00
2,2015-06-11 10:52:00,11.8,11.8,11.8,11.8,0.0,DELHIVERY,1,2015-06-11 10:50:00,2025-07-25 15:29:00
3,2015-06-11 10:53:00,11.8,11.8,11.8,11.8,0.0,DELHIVERY,1,2015-06-11 10:50:00,2025-07-25 15:29:00
4,2015-06-11 10:54:00,11.8,11.8,11.8,11.8,0.0,DELHIVERY,1,2015-06-11 10:50:00,2025-07-25 15:29:00


In [6]:
import os

save_dir = "../Data_processed"
os.makedirs(save_dir, exist_ok=True)

processed_path = os.path.join(save_dir, "final_processed.pkl")
final_df.to_pickle(processed_path)

print("Saved to:", processed_path)

Saved to: ../Data_processed/final_processed.pkl


#### 1. Basic setup & inspect data
Fill missing minutes for that company.
Add returns, moving averages, volatility.
Append to processed_list.


In [1]:
import pandas as pd

processed_path = "../Data_processed/final_processed.pkl"
final_df = pd.read_pickle(processed_path)

final_df.head()

,date,open,high,low,close,volume,company_name,company_number,company_start_timestamp,company_end_timestamp
0,2015-06-11 10:50:00,11.8,11.8,11.8,11.8,0.0,DELHIVERY,1,2015-06-11 10:50:00,2025-07-25 15:29:00
1,2015-06-11 10:51:00,11.8,11.8,11.8,11.8,0.0,DELHIVERY,1,2015-06-11 10:50:00,2025-07-25 15:29:00
2,2015-06-11 10:52:00,11.8,11.8,11.8,11.8,0.0,DELHIVERY,1,2015-06-11 10:50:00,2025-07-25 15:29:00
3,2015-06-11 10:53:00,11.8,11.8,11.8,11.8,0.0,DELHIVERY,1,2015-06-11 10:50:00,2025-07-25 15:29:00
4,2015-06-11 10:54:00,11.8,11.8,11.8,11.8,0.0,DELHIVERY,1,2015-06-11 10:50:00,2025-07-25 15:29:00


In [4]:
import pandas as pd
import numpy as np


In [6]:
companies = final_df["company_name"].unique()
price_cols = ["open", "high", "low", "close"]
volume_cols = ["volume"]

processed_list = []

for i, cname in enumerate(companies, start=1):
    print(f"Processing {i}/{len(companies)}: {cname}")

    df_c = final_df[final_df["company_name"] == cname].copy()
    df_c = df_c.sort_values("date")

    # 1) drop duplicate timestamps for this company
    #    (keep the first; change to 'last' if you prefer)
    before = len(df_c)
    df_c = df_c.drop_duplicates(subset="date", keep="first")
    after = len(df_c)
    if before != after:
        print(f"  Dropped {before - after} duplicate rows for {cname}")

    # 2) Reindex to 1‑minute frequency
    full_index = pd.date_range(
        start=df_c["date"].min(),
        end=df_c["date"].max(),
        freq="T"
    )
    df_c = df_c.set_index("date").reindex(full_index)

    df_c["missing_generated"] = df_c["company_name"].isna()

    # Forward-fill prices
    df_c[price_cols] = df_c[price_cols].ffill()

    # New rows: set volume to 0
    df_c.loc[df_c["missing_generated"], volume_cols] = 0

    # Forward-fill meta columns
    df_c["company_name"] = df_c["company_name"].ffill()
    df_c["company_number"] = df_c["company_number"].ffill()

    # Reset index back to date column
    df_c = df_c.reset_index().rename(columns={"index": "date"})

    # Feature engineering
    df_c = df_c.sort_values("date")

    df_c["log_return_1m"] = np.log(df_c["close"]) - np.log(df_c["close"].shift(1))
    df_c["ma_5"] = df_c["close"].rolling(window=5, min_periods=1).mean()
    df_c["ma_15"] = df_c["close"].rolling(window=15, min_periods=1).mean()
    df_c["log_return_5m"] = np.log(df_c["close"]) - np.log(df_c["close"].shift(5))
    df_c["log_return_15m"] = np.log(df_c["close"]) - np.log(df_c["close"].shift(15))
    df_c["vol_15m"] = df_c["log_return_1m"].rolling(window=15, min_periods=5).std()
    df_c["vol_60m"] = df_c["log_return_1m"].rolling(window=60, min_periods=10).std()

    processed_list.append(df_c)

final_processed_df = pd.concat(processed_list, ignore_index=True)
final_processed_df = final_processed_df.sort_values(["company_name", "date"]).reset_index(drop=True)

Processing 1/486: DELHIVERY


/var/folders/wv/ygt32m016zv4b7r2wb1yllnm0000gn/T/ipykernel_44524/4107479790.py:22: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  full_index = pd.date_range(
/Users/dnikita/Library/Python/3.9/lib/python/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/Users/dnikita/Library/Python/3.9/lib/python/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/Users/dnikita/Library/Python/3.9/lib/python/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


Processing 2/486: ATGL


/var/folders/wv/ygt32m016zv4b7r2wb1yllnm0000gn/T/ipykernel_44524/4107479790.py:22: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  full_index = pd.date_range(


Processing 3/486: TRENT


...

Processing 255/486: WELCORP


/var/folders/wv/ygt32m016zv4b7r2wb1yllnm0000gn/T/ipykernel_44524/4107479790.py:22: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  full_index = pd.date_range(


Processing 256/486: GILLETTE


/var/folders/wv/ygt32m016zv4b7r2wb1yllnm0000gn/T/ipykernel_44524/4107479790.py:22: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  full_index = pd.date_range(


: 

In [1]:
processed_list
save_dir = "../Data_processed"
os.makedirs(save_dir, exist_ok=True)

processed_path = os.path.join(save_dir, "processed_list.pkl")
processed_list.to_pickle(processed_path)

print("Saved to:", processed_list)

NameError: name 'processed_list' is not defined

#### 2. Ensure consistent timezone